In [237]:
import json
import time
from datetime import date
import pprint
from os import listdir
from feature_extract import feature_extract

HOUR_INDEX = 3
MINUTE_INDEX = 4
SECOND_INDEX = 5
    
files = listdir('../tweet_data')
print files

from os.path import splitext

import re

for fname in files:
    m = re.search('#.+', splitext(files[0])[0])
    if m:
        hashtag = m.group()
        print "Processing '%s'" % hashtag
    outfile = splitext(fname)[0] + '.csv'
    print "Output file name '%s'" % outfile
    
    f_handle = open('../tweet_data/'+fname)
    feature_extract(f_handle, outfile)
    f.close()
    break;

['tweets_#gohawks.txt', 'tweets_#gopatriots.txt', 'tweets_#nfl.txt', 'tweets_#patriots.txt', 'tweets_#sb49.txt', 'tweets_#superbowl.txt']
Processing '#gohawks'
Output file name 'tweets_#gohawks.csv'


NameError: global name 'json' is not defined

In [226]:
f = open('../tweet_data/tweets_#gohawks.txt')
line = f.readline()
tweet = json.loads(line)

tweet_time = tweet['firstpost_date']
start_time = list(time.localtime(tweet['firstpost_date']))

from datetime import date
start_date = date.fromtimestamp(float(tweet_time))
test = date(2014, 12, 30)
test > start_date

True

In [233]:
import json
from datetime import date

def feature_extract(f, o_file):
    line = f.readline()
    tweet = json.loads(line)

    n_tweets = 0 # number of tweets per hour
    n_followers = {} # number of followers of users posting the tweets per hour
    n_retweets = 0 # number of retweets per hour
    num_window = 0 # total number of hours

    user_follower = {}

    # outputs
    import numpy as np

    """
    output: n_tweets  n_retweets  sum_followers  max_followers  hour
    """
    output = np.empty([0, 5], dtype='int')

    # get start hour from first tweet
    start_time = list(time.localtime(tweet['firstpost_date']))
    start_time[MINUTE_INDEX] = 0; start_time[SECOND_INDEX] = 0
    nth_hour = start_time[HOUR_INDEX]
    start_time = time.mktime(start_time)
    start_date = date.fromtimestamp(start_time)
    print start_date
    print "Starting from time %s" % time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time))
    return
    time_format = "%Y-%m-%d %H:%M:%S"

    while len(line) != 0:
        tweet = json.loads(line)
        line = f.readline()
        tweet_time = tweet['firstpost_date']
        tweet_hour = time.localtime(tweet_time).tm_hour
        if tweet_hour != nth_hour or start_date != date.fromtimestamp(float(tweet_time)):
            print "Finished processing %d tweets from hour %d, next date time %s" %\
                  (n_tweets, nth_hour, time.strftime(time_format, time.localtime(tweet_time)))
            # update output
            new_hour = np.array([[n_tweets, n_retweets, sum(n_followers), max(n_followers), nth_hour]], dtype='int')
            output = np.append(output, new_hour, axis=0)
            # clear & update hourly variables
            num_window += 1
            n_tweets = 0 # number of tweets per hour
            n_followers = {} # number of followers of users posting the tweets per hour
            n_retweets = 0 # number of retweets per hour
            nth_hour = tweet_hour  # hour of the day

        # extract features
        n_tweets += 1
        n_retweets += tweet['tweet']['retweet_count']
        n_followers[tweet['tweet']['user']['id']] = tweet['tweet']['user']['followers_count']
        user_follower[tweet['tweet']['user']['id']] = tweet['tweet']['user']['followers_count']
        start_date = date.fromtimestamp(float(tweet_time))

    print "Outputting to '%s' ..." % o_file
    with open('data/' + o_file, 'w') as of:
        np.savetxt(o_file, output, delimiter=',')

    print "--------------------"
    print "Total number of tweets %d" % int(np.sum(output[:, 0]))
    print "Average number of tweets per hour %f" % np.mean(output[:, 0])
    print "Average number of retweets %f" % float(np.sum(output[:, 1])) / np.sum(output[:, 0])
    print "Average number of followers of users (%d) %f" %\
          (len(user_follower), float(sum(user_follower.values()) / float(len(user_follower))))

